In [1]:
# %%  
# Cell 1: Imports & Global Configuration

import os, glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, SparsePCA, TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
from operator import truediv
import joblib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

# Paths
PROCESSED_DIR = 'prepocessing/processed_cubes'
FIG_DIR       = 'figures'
RESULT_DIR    = 'results'
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

# Hyperparameters
WS        = 8        # patch size
NC        = 15        # number of spectral components
DLM       = 'PCA'       # PCA dimensionality limit multiplier
trRatio   = 0.05
vrRatio   = 0.05     # note: tr+vr+te should sum to 1; teRatio computed below
teRatio   = 0.90
randomState = 345

batch_size = 56
epochs     = 50
lr         = 1e-3

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

Using device: cuda


In [2]:
# %%  
# Cell 2: Load Preprocessed Cubes & Labels

def LoadHSIData(data_dir):
    files = glob.glob(os.path.join(data_dir, "*.npz"))
    species = sorted({os.path.basename(f).split('_block')[0] for f in files})
    label_map = {sp:i for i,sp in enumerate(species)}
    cubes, labels, names = [], [], []
    for f in files:
        sp = os.path.basename(f).split('_block')[0]
        arr = np.load(f)['block']  # (H, W, C)
        cubes.append(arr)
        labels.append(label_map[sp])
        names.append(sp)
    return cubes, np.array(labels), names, species

cubes, cube_labels, cube_names, species_list = LoadHSIData(PROCESSED_DIR)
num_classes = len(species_list)
print(f"Loaded {len(cubes)} blocks from {num_classes} species")


Loaded 26 blocks from 9 species


In [3]:
# %%  
# Cell 3: Plot Mean Spectral Signatures per Species

mean_spectra = {sp:[] for sp in species_list}
for arr, name in zip(cubes, cube_names):
    mean_spectra[name].append(arr.mean(axis=(0,1)))  # mean spectrum

plt.figure(figsize=(8,6))
for sp in species_list:
    spec = np.stack(mean_spectra[sp], axis=0).mean(axis=0)
    plt.plot(spec, label=sp)
plt.title("Mean Spectral Signature by Species")
plt.xlabel("Band Index")
plt.ylabel("Mean Reflectance")
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, "mean_spectra.png"))
plt.close()


In [4]:
# Sanity check: make sure we have blocks to process
print("Looking in:", PROCESSED_DIR)
files = glob.glob(os.path.join(PROCESSED_DIR, "*.npz"))
print("Found .npz files:", files[:5], "... total:", len(files))
if len(files) == 0:
    raise RuntimeError(f"No .npz files in {PROCESSED_DIR}. Check your path.")


Looking in: prepocessing/processed_cubes
Found .npz files: ['prepocessing/processed_cubes/sapelimahonki_block0.npz', 'prepocessing/processed_cubes/afromasia_block0.npz', 'prepocessing/processed_cubes/tiiki_block1.npz', 'prepocessing/processed_cubes/ipe_block0.npz', 'prepocessing/processed_cubes/tiiki_block0.npz'] ... total: 26


In [5]:
# %%  
# Cell 4 (Generalized): Global Dimensionality Reduction Fit & Apply

import joblib
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, SparsePCA, TruncatedSVD

# 1) Aggregate all pixel spectra
all_pixels = np.vstack([arr.reshape(-1, arr.shape[2]) for arr in cubes])
print("Aggregated pixels:", all_pixels.shape)

# 2) Choose and fit the DR model once, based on DLM
if DLM == 'PCA':
    dr_model = PCA(n_components=NC, whiten=True, random_state=0)
elif DLM == 'iPCA':
    dr_model = IncrementalPCA(n_components=NC)
elif DLM == 'KPCA':
    dr_model = KernelPCA(kernel='rbf', n_components=NC,
                        fit_inverse_transform=True, random_state=0)
elif DLM == 'SPCA':
    dr_model = SparsePCA(n_components=NC, alpha=1e-4, random_state=0)
elif DLM == 'SVD':
    dr_model = TruncatedSVD(n_components=NC, random_state=0)
else:
    raise ValueError(f"Unknown DLM: {DLM}")

# If iPCA, do a partial fit in batches
if DLM == 'iPCA':
    for batch in np.array_split(all_pixels, 256):
        dr_model.partial_fit(batch)
else:
    dr_model.fit(all_pixels)

# 3) Save the DR model
joblib.dump(dr_model, os.path.join(RESULT_DIR, f"{DLM}_model.joblib"))
print(f"Fitted {DLM}: input bands → {NC} components.")

# 4) Apply the same transformation to each cube
reduced_cubes = []
for arr in cubes:
    H, W, B = arr.shape
    flat = arr.reshape(-1, B)
    red  = dr_model.transform(flat)
    reduced_cubes.append(red.reshape(H, W, NC))

print(f"Applied {DLM} to all cubes.")


Aggregated pixels: (1162087, 256)
Fitted PCA: input bands → 15 components.
Applied PCA to all cubes.


In [ ]:
# %%  
# Cell 5: Create Spatial-Spectral Patches & Labels

def ImageCubes(HSI_list, labels, WS=14):
    patches, patch_labels = [], []
    for HSI, lab in zip(HSI_list, labels):
        H, W, NB = HSI.shape
        pad = WS//2
        padded = np.pad(HSI, ((pad,pad),(pad,pad),(0,0)), mode='constant')
        for i in range(pad, pad+H):
            for j in range(pad, pad+W):
                cube = padded[i-pad:i+pad, j-pad:j+pad, :]
                patches.append(cube)
                patch_labels.append(lab)
    patches = np.stack(patches, axis=0)              # (N, WS, WS, NC)
    patch_labels = np.array(patch_labels)            # (N,)
    return patches, patch_labels

patches, patch_labels = ImageCubes(reduced_cubes, cube_labels, WS)
print(f"Created {patches.shape[0]} patches of shape {patches.shape[1:]}")

Created 1162087 patches of shape (8, 8, 15)


In [ ]:
# %%  
# Cell 6: Train/Val/Test Split & DataLoaders

def TrTeSplit(X, y, trRatio, vrRatio, teRatio, rs=345):
    X_trte, X_te, y_trte, y_te = train_test_split(X, y, test_size=teRatio,
                                                  random_state=rs, stratify=y)
    vr = vrRatio/(trRatio+vrRatio)
    X_tr, X_va, y_tr, y_va = train_test_split(X_trte, y_trte, test_size=vr,
                                              random_state=rs, stratify=y_trte)
    return X_tr, X_va, X_te, y_tr, y_va, y_te

X_tr, X_va, X_te, y_tr, y_va, y_te = TrTeSplit(patches, patch_labels,
                                               trRatio, vrRatio, teRatio, randomState)

# convert to tensors and dataloaders
def to_loader(X, y, batch_size, shuffle=True):
    # X: (N, WS, WS, NC) → (N, NC, WS, WS)
    X = np.transpose(X, (0,3,1,2)).astype(np.float32)
    ds = TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=2)

train_dl = to_loader(X_tr, y_tr, batch_size)
val_dl   = to_loader(X_va, y_va, batch_size, shuffle=False)
test_dl  = to_loader(X_te, y_te, batch_size, shuffle=False)

print(f"Splits → Train: {len(X_tr)}, Val: {len(X_va)}, Test: {len(X_te)}")

Splits → Train: 58104, Val: 58104, Test: 1045879


In [8]:
# %%  
# Cell 7: 3D-CNN Definition

class Simple3DCNN(nn.Module):
    def __init__(self, num_classes, in_bands):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv3d(1,16,(5,3,3),padding=(2,1,1)), nn.ReLU(),
            nn.MaxPool3d((2,2,2)),
            nn.Conv3d(16,32,(3,3,3),padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool3d((1,1,1))
        )
        self.fc = nn.Linear(32, num_classes)
    def forward(self,x):
        x = x.unsqueeze(1)  # B,1,NB,WS,WS
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

model = Simple3DCNN(num_classes=num_classes, in_bands=NC).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=lr)


/home/hassaan/miniconda3/envs/seg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# %%  
# Cell 8: Training Loop with Metrics Tracking

train_losses, val_losses = [], []
train_accs,  val_accs   = [], []

for ep in range(1, epochs+1):
    # training
    model.train()
    tloss, tcorrect, ttotal = 0, 0, 0
    for x,y in train_dl:
        x,y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x)
        loss   = F.cross_entropy(logits, y)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        preds = logits.argmax(1)
        tloss   += loss.item()*y.size(0)
        tcorrect+= (preds==y).sum().item()
        ttotal  += y.size(0)
    train_losses.append(tloss/ttotal)
    train_accs.append(tcorrect/ttotal)

    # validation
    model.eval()
    vloss, vcorrect, vtotal = 0,0,0
    with torch.no_grad():
        for x,y in val_dl:
            x,y = x.to(DEVICE), y.to(DEVICE)
            logits = model(x)
            loss  = F.cross_entropy(logits, y)
            preds = logits.argmax(1)
            vloss   += loss.item()*y.size(0)
            vcorrect+= (preds==y).sum().item()
            vtotal  += y.size(0)
    val_losses.append(vloss/vtotal)
    val_accs.append(vcorrect/vtotal)

    print(f"Epoch {ep}/{epochs}  Train: loss={train_losses[-1]:.4f}, acc={train_accs[-1]:.3f}  |  "
          f"Val: loss={val_losses[-1]:.4f}, acc={val_accs[-1]:.3f}")


Epoch 1/50  Train: loss=0.7765, acc=0.753  |  Val: loss=0.3244, acc=0.903
Epoch 2/50  Train: loss=0.2471, acc=0.926  |  Val: loss=0.2826, acc=0.909
Epoch 3/50  Train: loss=0.1855, acc=0.943  |  Val: loss=0.1789, acc=0.945
Epoch 4/50  Train: loss=0.1533, acc=0.953  |  Val: loss=0.1335, acc=0.958
Epoch 5/50  Train: loss=0.1397, acc=0.957  |  Val: loss=0.1227, acc=0.963
Epoch 6/50  Train: loss=0.1239, acc=0.961  |  Val: loss=0.1334, acc=0.958
Epoch 7/50  Train: loss=0.1145, acc=0.964  |  Val: loss=0.1297, acc=0.959
Epoch 8/50  Train: loss=0.1084, acc=0.965  |  Val: loss=0.1230, acc=0.963
Epoch 9/50  Train: loss=0.1024, acc=0.967  |  Val: loss=0.0950, acc=0.970
Epoch 10/50  Train: loss=0.0968, acc=0.969  |  Val: loss=0.0890, acc=0.971
Epoch 11/50  Train: loss=0.0904, acc=0.971  |  Val: loss=0.0811, acc=0.975
Epoch 12/50  Train: loss=0.0882, acc=0.971  |  Val: loss=0.1057, acc=0.967
Epoch 13/50  Train: loss=0.0829, acc=0.973  |  Val: loss=0.0964, acc=0.969
Epoch 14/50  Train: loss=0.0808, a

In [10]:
# %%  
# Cell 9: Plot Loss and Accuracy Curves

fig, axs = plt.subplots(1,2, figsize=(12,5))
axs[0].plot(train_losses, label='Train'); axs[0].plot(val_losses, label='Val')
axs[0].set_title("Loss"); axs[0].legend()
axs[1].plot(train_accs, label='Train'); axs[1].plot(val_accs, label='Val')
axs[1].set_title("Accuracy"); axs[1].legend()
for ax in axs: ax.set_xlabel("Epoch")
plt.savefig(os.path.join(FIG_DIR, "train_val_curves.png"))
plt.close(fig)


In [11]:
# %%  
# Cell 10: Final Evaluation & Confusion Matrix

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for x,y in test_dl:
        x = x.to(DEVICE)
        p = model(x).argmax(1).cpu().numpy()
        all_preds.append(p)
        all_labels.append(y.numpy())
all_preds  = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Classification report
report = classification_report(all_labels, all_preds, target_names=species_list)
cm     = confusion_matrix(all_labels, all_preds)
kappa  = cohen_kappa_score(all_labels, all_preds)

print(report)
print(f"Overall Accuracy: {accuracy_score(all_labels, all_preds):.3f}")
print(f"Cohen's Kappa: {kappa:.3f}")

# Plot confusion matrix
plt.figure(figsize=(8,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=species_list, yticklabels=species_list)
plt.xlabel("Predicted"); plt.ylabel("Actual");
plt.title("Confusion Matrix")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, "confusion_matrix.png"))
plt.close()


               precision    recall  f1-score   support

       abachi       0.95      0.99      0.97    132565
    afromasia       0.98      1.00      0.99     83214
          ipe       0.99      0.95      0.97    129307
        iroko       1.00      1.00      1.00     86546
       merbau       0.99      0.99      0.99    119147
      ovangol       0.98      0.99      0.99    118237
       padauk       0.99      0.99      0.99    119854
sapelimahonki       0.98      0.99      0.98    116465
        tiiki       0.99      0.96      0.97    140544

     accuracy                           0.98   1045879
    macro avg       0.98      0.98      0.98   1045879
 weighted avg       0.98      0.98      0.98   1045879

Overall Accuracy: 0.983
Cohen's Kappa: 0.981
